# Import
---

#### Overview
Import image data to render-ws.


In [1]:
# indirectly enable autocomplete
%config Completer.use_jedi = False

# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
from itertools import chain
from natsort import natsorted
from tqdm.notebook import tqdm

from scripted_render_pipeline import basic_auth
from scripted_render_pipeline.importer import render_specs, uploader
from scripted_render_pipeline.importer.fastem_mipmapper import FASTEM_Mipmapper

#### `render-ws` environment variables
* `host` : Web address which hosts render-ws. It's usually the preamble of the link to render-ws html page, i.e. `{host_name}/render-ws/view/index.html`
* `owner` : render-ws ID of dataset
* `project`: desired project name
* `auth`: authorization for render-ws. See https://github.com/hoogenboom-group/scripted-render-pipeline/tree/master for instructions

In [3]:
# render-ws environment variables
params_render = {
    "host": "https://sonic.tnw.tudelft.nl",
    "owner": "fastem",
    "project": "20230914_RP_exocrine_partial_test",
    "auth": basic_auth.load_auth()
}

#### Import parameters

* `dir_project` : path to project directory -- assumes subdirectories are the ROA's (which are further assumed to be sections)
* `project_path` : path to ROA directory -- assumes contents are (a grid of) tiff files
* `parallel`: how many threads to use in parallel, optimises io usage
* `clobber`: wether to allow overwriting of existing mipmaps
* `mipmap_path`: where to save mipmaps, defaults to `{project_path}/_mipmaps`

Assumes data for every ROA (section) is post-corrected and in subfolders (`/postcorrection`). If using the raw data:  
`fps_ROA = natsorted([fp for fp in dir_project.glob("*") if fp.is_dir()])`

In [11]:
# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/tests/20231107_MCF7_UAC_test/")

# set parameters for importing
params_import = {
    "parallel": 40,
    "clobber": True,
    "mipmap_path": dir_project / "_mipmaps/"
}

# collect filepaths to ROA's
fps_ROA = natsorted([fp for fp in dir_project.glob("*/postcorrection/") if fp.is_dir()])
fps_ROA = {k: v for k, v in enumerate(fps_ROA)}
fps_ROA

{0: PosixPath('/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S054/postcorrection'),
 1: PosixPath('/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S055/postcorrection'),
 2: PosixPath('/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S056/postcorrection')}

#### Create mipmaps
Create mipmaps (Image pyramids) of every `field` image. `mipmapper.create_all_mipmaps()` creates a new subfolder in `dir_project` named `_mipmaps`, and nested subfolders for each z level (zero-indexed) and field image:  

```
.
└── _mipmaps/
    ├── 0/
    │   ├── 000x000/
    │   │   ├── 0.tif
    │   │   ├── 1.tif
    │   │   ├── 2.tif
    │   │   ├── 3.tif
    │   │   ├── 4.tif
    │   │   ├── 5.tif
    │   │   ├── 6.tif
    │   │   ├── 7.tif
    │   │   └── 8.tif
    │   ├── 000x001
    │   └── 000x002
    ├── 1/
    │   ├── 000x000
    │   ├── 000x001
    │   └── 000x002
    └── 2/
```

In [1]:
# create mipmaps from mapping of z values to ROA/section directories
mipmapper = FASTEM_Mipmapper(
    project_paths=fps_ROA,
    **params_import
)
stacks = mipmapper.create_all_mipmaps()

#### Import to render-ws
Import `stack` object to `render-ws`
* `z_resolution`: section thickness, has to be set because this cannot be inferred from the metadata.

In [ ]:
# initialize uploader
uppity = uploader.Uploader(
    **params_render,
    clobber=params_import.get("clobber")
)

# import stack to render-ws
uppity.upload_to_render(
    stacks=stacks,
    z_resolution=80
)

uploading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.44stacks/s]


#### Plot stacks
Plot imported stacks to inspect

In [7]:
from interactive_render import plotting

In [8]:
# plot stack
plotting.plot_stacks(
    stacks=[stack.name for stack in stacks],
    render=uppity.render
)

  0%|          | 0/1 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=16), IntSlider(value=30207, description='vmin', …